<a href="https://colab.research.google.com/github/harika140901/ee5332-2025/blob/main/misc/Simplex_single_phase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install mip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 28.1 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.17.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


In [1]:
import numpy as np

def print_tableau(A, b, c, f):
    for i in range(len(A)):
        print("{0:30} | {1}".format(str(A[i]), b[i]))
    print('_______________________________________')
    print("{0:30} | {1}\n".format(str(c), f))

def simplex(m, obj, verbose=False):
    varMap = {v.name: i for i, v in enumerate(m.vars)}
    numVars = len(m.constrs) + len(m.vars)
    A = np.zeros(shape=(len(m.constrs), numVars))
    c = np.zeros(numVars)

    # Objective function
    for i, v in obj.expr.items():
        c[varMap[i.name]] = v

    b = np.zeros(len(m.constrs))

    # Constraints processing
    for i, e in enumerate(m.constrs):
        sens = e.expr.sense
        b[i] = e.rhs if sens == '<' else -e.rhs
        for ind, val in e.expr.expr.items():
            A[i][varMap[ind.name]] = val if sens == '<' else -val
        if sens == '<' or sens == '>':
            A[i][len(m.vars) + i] = 1

    f = 0
    sol = np.zeros(numVars)

    print("Initial tableau:")
    print_tableau(A, b, c, f)

    # Simplex Iterations
    while np.any(c > 0):  # While there are positive coefficients in the objective row
        pivot_col = np.argmax(c)  # Entering variable (most positive coefficient)

        # Select pivot row
        ratios = []
        for i in range(len(b)):
            if A[i][pivot_col] > 0:
                ratios.append(b[i] / A[i][pivot_col])
            else:
                ratios.append(float('inf'))  # Avoid division by zero

        pivot_row = np.argmin(ratios)  # Leaving variable

        if ratios[pivot_row] == float('inf'):
            raise ValueError("Unbounded solution")  # No valid pivot row

        # Pivoting
        pivot_val = A[pivot_row][pivot_col]
        A[pivot_row] /= pivot_val  # Normalize pivot row
        b[pivot_row] /= pivot_val

        for i in range(len(A)):
            if i != pivot_row:
                factor = A[i][pivot_col]
                A[i] -= factor * A[pivot_row]
                b[i] -= factor * b[pivot_row]

        factor = c[pivot_col]
        c -= factor * A[pivot_row]
        f -= factor * b[pivot_row]

        if verbose:
            print("\nTableau after pivoting:")
            print_tableau(A, b, c, f)

    # Extract solution
    for i in range(len(m.vars)):
        for j in range(len(A)):
            if A[j][i] == 1 and sum(A[:, i]) == 1:
                sol[i] = b[j]
                break

    print("\nFinal tableau:")
    print_tableau(A, b, c, f)

    print("Optimal solution:", sol)
    print("Optimal value:", f, "\n")

    return sol, f

In [ ]:
import numpy as np
def print_tableau(A, b, c, f):
  for i in range(len(A)):
    print("{0:30} | {1}".format(str(A[i]), b[i]))
  print('_______________________________________')
  print("{0:30} | {1}\n".format(str(c), f))

def simplex(m, obj, verbose = False):
    varMap = {v.name:i for i,v in enumerate(m.vars)}
    numVars = len(m.constrs) + len(m.vars)
    A = np.zeros(shape=(len(m.constrs), numVars))
    c = np.zeros(numVars)
    for i, v in obj.expr.items():
        c[varMap[i.name]] = v
    b = np.zeros(len(m.constrs))
    for i, e in enumerate(m.constrs):
        sens = e.expr.sense
        b[i] = e.rhs if sens == '<' else -e.rhs
        for ind,val in e.expr.expr.items():
            A[i][varMap[ind.name]] = val if sens == '<' else -val
        if sens == '<' or sens == '>': A[i][len(m.vars) + i] = 1
    f = 0
    sol = np.zeros(numVars)

    print("Initial tableau:")
    print_tableau(A, b, c, f)

    ### fill in the simplex core

    print("Final tableau:")
    print_tableau(A, b, c, f)

    print("solution : ", sol, "\n")
    return sol



In [4]:
from mip import Model, maximize

m = Model("test")
x = [m.add_var(), m.add_var()]
obj = m.objective = maximize(x[0] + x[1])
m += x[0] +  3 * x[1] <= 9
m += 2 * x[0] + x[1] <= 8
simplex(m, obj)


m = Model("test")
x = [m.add_var() for i in range(3)]
obj = m.objective = maximize(6 * x[0] + x[1])
m += 9 * x[0] + x[1] + x[2] <= 18
m += 24 * x[0] + x[1] + 4 * x[2] <= 42
simplex(m, obj)

Initial tableau:
[1. 3. 1. 0.]                  | 9.0
[2. 1. 0. 1.]                  | 8.0
_______________________________________
[1. 1. 0. 0.]                  | 0


Final tableau:
[ 0.   1.   0.4 -0.2]          | 2.0
[ 1.   0.  -0.2  0.6]          | 3.0
_______________________________________
[ 0.   0.  -0.2 -0.4]          | -5.0

Optimal solution: [3. 2. 0. 0.]
Optimal value: -5.0 

Initial tableau:
[9. 1. 1. 1. 0.]               | 18.0
[24.  1.  4.  0.  1.]          | 42.0
_______________________________________
[6. 1. 0. 0. 0.]               | 0


Final tableau:
[9. 1. 1. 1. 0.]               | 18.0
[15.  0.  3. -1.  1.]          | 24.0
_______________________________________
[-3.  0. -1. -1.  0.]          | -18.0

Optimal solution: [ 0. 18.  0.  0.  0.]
Optimal value: -18.0 



(array([ 0., 18.,  0.,  0.,  0.]), np.float64(-18.0))